# 繼承

### 在面相對象編輯中,當已經創建了一個類,然又想再創建一個相似的類,例如添加幾個方法,或者修改原來的方法,此時不須從頭開始,可以從原來的類生成一個新的類,原本的類稱為''父類''或''基類'',而生成的類稱為''子類'',子類繼承了父類所有的數據與方法

In [1]:
class Animal(object):
    def __init__(self, name):
        self.name = name
    def greet(self):
        print('Hello, I am %s.' % self.name)
class Dog(object):
    def __init__(self, name):
        self.name = name
    def greet(self):
        print('WangWang.., I am %s. ' % self.name)

### 可以看到，Dog類和Animal類幾乎是一樣的，只是greet方法不一樣，我們完全沒必要創建一個新的類，而是從Animal類派生出一個新的類：

In [5]:
class Animal(object):
    def __init__(self,name):
        self.name = name
    def greet(self):
        print('Hello, I am %s'%self.name)
class Dog(Animal):
    def greet(self): #對父類方法進行修改
        print('wangwang...,i am %s' % self.name)
    def run(self):   #添加新的方法
        print('i am running~')

In [2]:
animal = Animal('animal') #創建animal實例
animal.greet()

Hello, I am animal


### Dog類是從Animal類繼承而來,Dog類自動獲得Animal類所有數據和方法,而且還可以對父類進行修改

In [4]:
dog = Dog('dog')
dog.greet()

wangwang...,i am dog


# 多態
### 多態的概念其實不難理解,它是
### 指'''對不同類型的變量進行相同的操作,他會根據對象(或類)類型的不同而表現出不同的行為'''
### 事實上,我們經常用到多態的性質 例如以下可以看到，我們對兩個整數進行+操作，會返回它們的和，對兩個字符進行相同的+操作，會返回拼接後的字符串。也就是說，不同類型的對像對同一消息會作出不同的響應。

In [2]:
print(1+2)
print('a'+'b')

3
ab


### 再看看類的例子

In [5]:
class Animal(object):
    def __init__(self,name):
        self.name = name
    def greet(self):
        print('Hello, I am %s.' % self.name)
        
class Dog(Animal):
    def greet(self):
        print('Wang Wang!, I am %s'%self.name)

class Cat(Animal):
    def greet(self):
        print('Miao~, I am %s'%self.name)
        
def hello(animal):
    animal.greet()

In [6]:
dog = Dog('dog')
print(hello(dog))
cat = Cat('cat')
print(hello(cat))

Wang Wang!, I am dog
None
Miao~, I am cat
None


### 可以看到，cat和dog是兩個不同的對象，對它們調用greet方法，它們會自動調用實際類型的greet方法，作出不同的響應。這就是多態的魅力。

# Super用法
### 在類的繼承中，如果重定義某個方法，該方法會覆蓋父類的同名方法，但有時，我們希望能同時實現父類的功能，這時，我們就需要調用父類的方法了，可通過使用super來實現，比如：

In [8]:
class Animal(object):
    def __init__(self, name):
        self.name = name
    def greet(self):
        print ('Hello, I am %s.' % self.name)

class Dog(Animal):
    def greet(self):
        super(Dog, self).greet()   # Python3 可使用 super().greet()
        print('WangWang...')

### 在上面，Animal是父類，Dog是子類，我們在Dog類重定義了greet方法，為了能同時實現父類的功能，我們又調用了父類的方法，看下面的使用：

In [9]:
dog = Dog('Dog')
dog.greet()

Hello, I am Dog.
WangWang...


### super 的一個最常見用法可以說是在子類中調用父類的初始化方法了，比如：

In [23]:
class Base(object):
    def __init__(self, a, b):
        self.a = a
        self.b = b

class A(Base):
    def __init__(self, a, b, c):
        super(A, self).__init__(a, b)  # Python3 可使用 super().__init__(a, b)
        self.c = c

In [24]:
A.mro()

[__main__.A, __main__.Base, object]

### 根據super工作原理, 根據mro列表依序檢查,因此可以說是在子類調用父類的初始化方法

# 深入super

### 看了上面的使用，你可能會覺得super的使用很簡單，無非就是獲取了父類，並調用父類的方法。其實，在上面的情況下，super獲得的類剛好是父類，但在其他情況就不一定了，super其實和父類沒有實質性的關聯。

In [19]:
class Base(object):
    def __init__(self):
        print ("enter Base")
        print ("leave Base")

class A(Base):
    def __init__(self):
        print ("enter A")
        super(A, self).__init__()
        print ("leave A")

class B(Base):
    def __init__(self):
        print ("enter B")
        super(B, self).__init__()
        print ("leave B")

class C(A, B):
    def __init__(self):
        print ("enter C")
        super(C, self).__init__()
        print ("leave C")

### 其中，Base 是父類，A, B 繼承自Base, C 繼承自A, B，它們的繼承關係是一個典型的『菱形繼承』，如下：
      Base
      /  \
     /    \
    A      B
     \    /
      \  /
       C

In [20]:
c = C()
print(c)

enter C
enter A
enter B
enter Base
leave Base
leave B
leave A
leave C


### 如果認為super代表『調用父類的方法』，那你很可能會疑惑為什麼enter A的下一句不是enter Base而是enter B。原因是，super和父類沒有實質性的關聯，現在讓我們搞清super是怎麼運作的。

# MRO列表
### 事實上，對於你定義的每一個類，Python會計算出一個方法解析順序（Method Resolution Order, MRO）列表，它代表了類繼承的順序，我們可以使用下面的方式獲得某個類的MRO列表：
### 那這個MRO列表的順序是怎麼定的呢，它是通過一個C3線性化算法來實現的，這裡我們就不去深究這個算法了，感興趣的讀者可以自己去了解一下，總的來說，一個類的MRO列表就是合併所有父類的MRO列表，並遵循以下三條原則：

### 1.子類永遠在父類前面
### 2.如果有多個父類，會根據它們在列表中的順序被檢查
### 3.如果對下一個類存在兩個合法的選擇，選擇第一個父類

In [22]:
C.mro()

[__main__.C, __main__.A, __main__.B, __main__.Base, object]

# Super　工作原理
def super(cls, inst):
    mro = inst.__class__.mro()
    return mro[mro.index(cls) + 1]
### 其中，cls 代表類，inst 代表實例，上面的代碼做了兩件事：

### 1.獲取inst 的MRO 列表
### 2.查找cls 在當前MRO 列表中的index, 並返回它的下一個類，即mro[index + 1]
### 當你使用super(cls, inst)時，Python會在inst的MRO列表上搜索cls的下一個類。

# 小結
### 繼承可以拿到父類的所有數據和方法，子類可以重寫父類的方法，也可以新增自己特有的方法。
### 有了繼承，才有了多態，不同類的對像對同一消息會作出不同的相應。
### 事實上，super和父類沒有實質性的關聯。
### super(cls, inst) 獲得的是cls 在inst 的MRO 列表中的下一個類。